In [1]:
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
import numpy as np
import pandas as pd
import torch

In [2]:
MODEL_NAME = "512-encoder-1000K/kaggle/working/results/checkpoint-125000/"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


T5ForConditionalGeneration(
  (shared): Embedding(64101, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64101, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [3]:
model.generation_config
start_token = "<unk>"
end_token = "</s>"

In [4]:
from datasets import load_dataset

dataset = load_dataset("mateiaassAI/MEID_v2", split=['train[:97%]', 'train[97%:100%]'])
dataset

[Dataset({
     features: ['wrong', 'right'],
     num_rows: 1967695
 }),
 Dataset({
     features: ['wrong', 'right'],
     num_rows: 60857
 })]

In [5]:
ds_train = dataset[0]
ds_test = dataset[1]
ds_test

Dataset({
    features: ['wrong', 'right'],
    num_rows: 60857
})

In [6]:
punctuation_marks = ['.', '?', '!', ';', '...']

def filter_sentences(sentences):
    text = sentences['right']
    if any(text.endswith(punc) for punc in punctuation_marks):
      words = text.split()
      if len(words) >= 10:
        return True
    return False

In [7]:
fds_test = ds_test.filter(filter_sentences, batched=False)

In [8]:
fds_test[0]
print(len(fds_test))

31457


In [ ]:
import json

def extract_text(generated_text, start_token="<unk>", end_token="</s>"):
    start_index = generated_text.find(start_token)
    end_index = generated_text.find(end_token, start_index)
    if start_index != -1 and end_index != -1:
        return generated_text[start_index + len(start_token):end_index].strip()
    else:
        return None

sentence_pairs = []
batch_index = 5

for idx, data in enumerate(fds_test):
    if idx < 5000:
        continue;
        
    print(idx)
    correct_sentence = data['right']
    input_sentence = data['wrong']

    input_tensor = tokenizer.encode(input_sentence, return_tensors="pt").to(device)
    outputs = model.generate(input_tensor)
    generated_text = tokenizer.decode(outputs[0].cpu(), skip_special_tokens=False)
    predicted_sentence = extract_text(generated_text)

    if predicted_sentence is not None:
        sentence_pairs.append(
            (correct_sentence, input_sentence, predicted_sentence))

    if len(sentence_pairs) >= 1000:
        df = pd.DataFrame(sentence_pairs)
        file_name = f'ress/sentence_pairs_batch_{batch_index}.jsonl'
        with open(file_name, 'w', encoding='utf-8') as f:
            for pair in sentence_pairs:
                json.dump(pair, f, ensure_ascii=False)
                f.write('\n')
        print(f"Saved 1000 sentence pairs to {file_name}")
        sentence_pairs = []  # Clear the list after saving
        batch_index += 1  # Increment the batch index

    if batch_index == 10:
        break

5000
5001
5002
5003
5004
5005
5006
5007
5008
5009
5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199


In [25]:
print(sentence_pairs[0][2])

Pentru perioada în care un pacient are deschisă o fișă de spitalizare de zi pe<unk> parcursul unei singure zile sau pe<unk> parcursul mai multor zile, acesta poate beneficia și de servicii medicale în ambulatoriul de specialitate, altele decât cele necesare acordării serviciilor medicale din spitalizare de zi, cu respectarea condițiilor de acordare a serviciilor medicale în ambulatoriu.


In [26]:
input_ids = tokenizer("Maria sa dus la mare ca sa mannce incchetata cu vanilie ", return_tensors="pt").input_ids
generated_text = tokenizer.decode(model.generate(input_ids)[0], skip_special_tokens=False)

start_index = generated_text.find(start_token)
end_index = generated_text.find(end_token, start_index)

if start_index != -1 and end_index != -1:
    extracted_text = generated_text[start_index + len(start_token):end_index].strip()
else:
    extracted_text = None

print(extracted_text)

Maria sa dus la mare ca sa manance incicheta cu vanilie.
